# Now that I have all the images downloaded

I can now see if I can use Optical Character Recognition to read text from cards. 

If I can I can first to extract the card name and then narrowing down the set of cards to compare in our vector database via cosine similarity. 

I want to utilize a pretrained model to embed the card images and store them in a vector database. A user will then upload a picture from their phone of a card and it will use the OCR to find text in the card and then narrow down the search in the database to only those cards. It will then use cosine similarity to compare the overall card images to find the exact match.

This will lead to a reduced search space.

In [35]:
import pytesseract  # Python library for OCR (Optical Character Recognition)
from PIL import Image # PIL is the Python Imaging Library, which provides the imaging capabilities for PILLOW

# Set the Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


def extract_card_name(image_path):
    # Load image with PIL
    img = Image.open(image_path)
    
    # Perform OCR on the image
    text = pytesseract.image_to_string(img)
    
    # Post-process the OCR result to extract the card name (e.g., via regex)
    # This assumes the card name appears at the top of the card
    card_name = text.splitlines()[0]  # Naive assumption that the first line is the card name
    
    return card_name.strip()

# def extract_all_text(image_path):
#     # Preprocess the image
#     processed_img = preprocess_image(image_path)
    
#     # Perform OCR with custom configuration
#     config = '--oem 3 --psm 3'  # OEM 3: Default, PSM 3: Fully automatic page segmentation
#     text = pytesseract.image_to_string(processed_img, config=config)
    
#     return text

def extract_all_text(image_path):
    # Load image with PIL
    img = Image.open(image_path)
    
    # Perform OCR on the image
    text = pytesseract.image_to_string(img)
    
    return text

## Results were strange so lets clean up the images

In [36]:
import cv2
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding to binarize the image
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    
    # Optional: Apply GaussianBlur to reduce noise
    blur = cv2.GaussianBlur(thresh, (3, 3), 0)
    
    # Convert back to PIL format for pytesseract
    processed_img = Image.fromarray(blur)
    
    return processed_img


In [37]:
print(extract_all_text('Datasets/mtg_images/000a7263-37e6-4246-82f9-94459517a5cc.jpg'))

Essence Warden

serene

Whenever another ci
battlefield, you gain 1

aera oar

"Life begets li

0126 / 1158
| PRM + EN 16 Goran Josie




## Trying to segment to image to only grab the card name

In [38]:
def segment_image(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    
    # Get the image dimensions
    height, width = img.shape[:2]
    
    # Define relative ROIs based on percentages of the image size
    card_name_roi = img[int(0.05 * height):int(0.15 * height), int(0.07 * width):int(0.8 * width)]  # Adjust percentages as needed
    
    # Convert ROIs to PIL format for pytesseract
    card_name_img = Image.fromarray(card_name_roi)
    
    return card_name_img

def extract_card_name_from_segment(image_path):
    # Segment the image into regions
    card_name_img = segment_image(image_path)
    
    # Perform OCR on the card name region
    config = '--oem 3 --psm 6'
    card_name_text = pytesseract.image_to_string(card_name_img, config=config)
    
    return card_name_text.strip()


print(extract_card_name_from_segment('Datasets/mtg_images/0000cd57-91fe-411f-b798-646e965eec37.jpg'))


Siren Lookout
